In [16]:
import torch
import re
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from datasets import Dataset
import pandas as pd

In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("datatattle/covid-19-nlp-text-classification")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'covid-19-nlp-text-classification' dataset.
Path to dataset files: /kaggle/input/covid-19-nlp-text-classification


In [10]:
file = "/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv"
df = pd.read_csv(file)
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [11]:
print("columns:", df.columns.tolist())

columns: ['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet', 'Sentiment']


In [12]:
df = df[['OriginalTweet', 'Sentiment']]
df.head()

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,When I couldn't find hand sanitizer at Fred Me...,Positive
2,Find out how you can protect yourself and love...,Extremely Positive
3,#Panic buying hits #NewYork City as anxious sh...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [13]:
label2id = {label: i for i, label in enumerate(df["Sentiment"].unique())}
id2label = {i: label for label, i in label2id.items()}
df["label"] = df["Sentiment"].map(label2id)

In [17]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

In [18]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [20]:
def tokenize(batch):
  return tokenizer(batch["OriginalTweet"], padding="max_length", truncation=True, max_length= 128)

train_ds = train_ds.map(tokenize)
test_ds = test_ds.map(tokenize)

Map:   0%|          | 0/3038 [00:00<?, ? examples/s]

Map:   0%|          | 0/760 [00:00<?, ? examples/s]

In [21]:
train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [22]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label2id), id2label= id2label, label2id= label2id)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
training_args= TrainingArguments(output_dir="./results", eval_strategy= "epoch", learning_rate= 1e-5, per_device_eval_batch_size= 16, per_device_train_batch_size= 16, num_train_epochs= 5, weight_decay= 0.01, logging_steps= 50)

In [25]:
def compute_metrics(pred):
    preds = pred.predictions.argmax(-1)
    labels = pred.label_ids
    acc = (preds == labels).mean()
    return {"accuracy": acc}

In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipython-input-1359719364.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: surabhi-pandey (surabhi-pandey-iit-guwahati) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,1.507100,1.380090,0.406579
2,1.192000,1.175321,0.536842
3,1.010200,1.082296,0.571053
4,0.920900,1.068561,0.576316
5,0.864400,1.045054,0.576316


TrainOutput(global_step=950, training_loss=1.121414465653269, metrics={'train_runtime': 301.5449, 'train_samples_per_second': 50.374, 'train_steps_per_second': 3.15, 'total_flos': 503071859673600.0, 'train_loss': 1.121414465653269, 'epoch': 5.0})

In [27]:
metrics = trainer.evaluate()
print(metrics)

# Predict new tweet
def predict_tweet(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(model.device)
    with torch.no_grad():
        logits = model(**inputs).logits
    pred_id = logits.argmax(dim=-1).item()
    return id2label[pred_id]

print(predict_tweet("I absolutely loved this update!"))
print(predict_tweet("This game makes me want to uninstall Steam."))


{'eval_loss': 1.0450537204742432, 'eval_accuracy': 0.5763157894736842, 'eval_runtime': 2.777, 'eval_samples_per_second': 273.672, 'eval_steps_per_second': 17.285, 'epoch': 5.0}
Extremely Positive
Neutral
